In [1]:
%cd ../../

/home/hoanghu/projects/Food-Waste-Optimization


In [6]:
import json
from pathlib import Path
from itertools import combinations

import numpy as np
import pandas as pd
from scipy.spatial import distance
from Levenshtein import ratio
from transformers import pipeline

In [3]:
path = "experiments_hoangle/hypotheses/res/6_dish_vs_customers/dish_names.json"

with open(path) as f:
    raw = json.load(f)

dish_names = list(raw.values())

## Measure Levenshtein distance

In [7]:
dists_leven = []
for pair in combinations(dish_names, 2):
    # print(f"{ratio(pair[0], pair[1]):.4f}: {pair[0]} <-> {pair[1]}")
    dists_leven.append({
        'word1': pair[0],
        'word2': pair[1],
        'dist_leven': ratio(pair[0], pair[1])
    })

df_dist_leven = pd.DataFrame.from_records(dists_leven)
df_dist_leven.head()

,word1,word2,dist_leven
0,Kalapuikot tillikermaviilikast,Uunimakkaraa,0.333333
1,Kalapuikot tillikermaviilikast,Marokkolainen linssipata,0.370370
2,Kalapuikot tillikermaviilikast,Herkkulohipihvit,0.304348
3,Kalapuikot tillikermaviilikast,Pasta Carbonara,0.266667
4,Kalapuikot tillikermaviilikast,Vegaaninen buttertofu,0.313725


In [9]:
df_dist_leven.sort_values('dist_leven', ascending=False).head(30)

,word1,word2,dist_leven
28889,Herkkusienikeitto,Herkkusienikeittoa,0.971429
10675,Falafelpyörykät,Falafel-pyörykät,0.967742
20812,Kikhernetaginea,Kikhernetagine,0.965517
6281,Chili-bataattikeittoa,Chilibataattikeitto,0.950000
15997,Jambalayaa,Jambalaya,0.947368
7259,Kanapyörykät,Kalapyörykät,0.916667
14669,Broileria pestokastikkeessa,Broileria pekonikastikkeessa,0.909091
5801,Karibian kasvispihvejä,Karibian kasvispihvi,0.904762
9323,Kultakuorrepyöryköitä,Kultakuorrepyörykät,0.900000
18757,Meksikolaista uunimakkaraa,Meksikolaista uunikalaa,0.897959


## Measure distance with language model's embeddings

In [4]:
pipeline = pipeline('feature-extraction', model='Finnish-NLP/gpt2-medium-finnish')


/home/hoanghu/miniforge3/envs/fw/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
def cosine_mat(mat: np.ndarray):
    tmp = mat / np.linalg.norm(mat, axis=-1, keepdims=True)

    cosine = 1 - tmp @ tmp.T

    return cosine


In [9]:
# dish_names = [
#     # 'Kalapuikot',
#     # 'Kalapuikot tillikermaviilikast'

#     # 'Jambalaya',
#     # 'Jambalayaa',

#     'Falafel-pyörykät',
#     'Falafelpyörykät',
#     'Falafelpyörylöitä',

#     # 'Butter Chicken',
#     # 'Butter fish',
# ]

output = pipeline(dish_names)


embds = np.float32([
    np.array(o).squeeze().sum(axis = 0)
    for o in output
])

In [11]:
sim = cosine_mat(embds)

In [32]:
THRES = 7E-3

pairs = np.argwhere(sim + np.triu(np.ones(embds.shape[0])) <= THRES)

for pair in pairs:
    print(f"{dish_names[pair[0]]} <-> {dish_names[pair[1]]}")

Aurajuusto-pinaattilasagnettea <-> Feta-pinaattilasagnette
Kirjolohipihvit <-> Kirjolohipyörykät
Kalapyörykät <-> Kanapyörykät
Kikhernetagine <-> Kikhernetaginea
Falafel-pyörykät <-> Falafelpyörykät
Kultakuorrepyörykät <-> Kultakuorrepyöryköitä
Lohimurekepihvit <-> Lohimurekepihvejä
Tempuraturskaa <-> Tempuraturskapaloja
Falafelpyörylöitä <-> Falafelpihvejä
Kanapyöryköitä <-> Kanapyörykät
Lohipyöryköitä <-> Kirjolohipyörykät
Jambalaya <-> Jambalayaa
